# Data Clustering $\def\*#1{\mathbf{#1}}$ $\DeclareMathOperator*{\argmax}{arg\,max}$ $\DeclareMathOperator*{\argmin}{arg\,min}$

*"Given a set of data points, partition them into groups containing very similar data points."* [Aggarwal, 2015]

This traditional **optimization problem** consists of partitioning a dataset, $D = \{\*x_i\}_{i=1}^n$, into $k$ groups called **custers** with respect to a given **objective function**. This resulting **clustering** is denoted by $\mathcal{C} = \{C_1, C_2,\ldots,C_k\}$.

Broadly speaking, this problem is known to be NP-Complete. Therefore, there is no polytnomial time algorithm for solving it unless $P$ is equal to $NP$. Basically, this means that there is **no known polynomial time algorithm** for solving it and that there is little chance of finding one. Theresefore, we usualy only consider **heuristics**, **meta-heuristics** or **approximation algorithms** for solving this optimization problem. Such methods, when used with care, can lead to find a *good* solution, but not necessearily an optimal solution.

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd

import sklearn.datasets as sk_data
import sklearn.metrics as metrics
from sklearn.cluster import KMeans

import scipy.stats as stats

import matplotlib.pyplot as plt

%matplotlib notebook

In [ ]:
n = 1000

cov = [[2, 0],
       [0, 2]]
mean = (1, 1)
D1 = np.random.multivariate_normal(mean, cov, n)

cov = [[10, 0],
       [0, 10]]
mean = (20, 10)
D2 = np.random.multivariate_normal(mean, cov, n)

cov = [[70, 30],
       [30, 20]]
mean = (0, 20)
D3 = np.random.multivariate_normal(mean, cov, n)

fig, ax = plt.subplots()
ax.plot(D1[:,0], D1[:,1], 'ob', D2[:,0], D2[:,1], 'or', D3[:,0], D3[:,1], 'og')

### Exhaustive Algorithm

In [ ]:
def S(n, k):
    """ Stirling numbers of the second kind """
    assert n >= 0 and k >= 0
    
    if k > n:
        return 0
    
    if n == 0 and k == 0:
        return 1
    
    if k == 0:
        return 0
    
    return k * S(n - 1, k) + S(n - 1, k - 1)

vS = np.vectorize(S)

k_values = range(0, 5)
n = 15
numbers = vS(n, k_values)

print(numbers)

fig, ax = plt.subplots()
ax.plot(numbers, color='red')
ax.set_ylabel(f'S({n}, k)')
ax.set_xlabel('k')
ax.set_xticks(range(0, 5))

## Representative Clustering

The purpose is to determine a set of $k$ representatives $\*y_1, \ldots, \*y_k$ that minimize the following **objective function** :

$$
\sum_{i = 1}^n \min\{d(\*x_i, \*y_j) : j = 1, \ldots, k\}
$$


### Data Clustering with k-means

* Select $k$ points as initial cluster centers $C_1, \ldots, C_k$.
* Repeat until convergence :
    * For $1 \leq i \leq n$, map point $p_i$ to its nearest cluster center $C_j$
    * Compute centroid $C_j'$ of the points nearest $C_j$ , for $1 \leq j \leq k$
    * For all $1 \leq j \leq k$, set $C_j = C_j'$

![The iterations of k-means (for k = 3)](figures/kmeans_iterations.png)

### Generating our data

In [ ]:
X, y = sk_data.make_blobs(n_samples=100, centers=4, n_features=2, center_box=(-10.0, 10.0), random_state=0, cluster_std=0.6)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X[:,0], X[:,1], c=y)

### Computing the pairwise distances for visualization purposes


We can compute pairwise distances using the **sklearn.metrics** functions summarized here:
http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

In [ ]:
euclidean_dists = metrics.euclidean_distances(X)

Visualizing the data using the heatmap of pairwise distances

In [ ]:
def plot_distance_matrix(D):
    fig, ax = plt.subplots()
    image = ax.matshow(D)
    fig.colorbar(image)
    ax.set_xticks([])
    ax.set_yticks([])
    fig.suptitle('Distance Matrix')
    
plot_distance_matrix(euclidean_dists)

### Clustering data using  k-means clustering with [sklearn.cluster ](http://scikit-learn.org/stable/modules/clustering.html)

In [ ]:
kmeans = KMeans(init='k-means++', n_clusters=4, n_init=1)
kmeans.fit_predict(X)
centroids = kmeans.cluster_centers_
labels = kmeans.labels_
error = kmeans.inertia_

print("The total error of the clustering is: ", error)
print('\nCluster Labels')
print(labels)
print('\nCluster Centroids')
print(centroids)

There are 3 functions in all the clustering classes, **fit()** **predict()** and **fit_predict()** :

* **fit()** is building the model from the training data (e.g. finding the centroids),             
* **predict()** is assigning labels to test data after building the model,           
* **fit_predict()** is doing both in the same data (e.g in kmeans, it finds the centroids and assigns the labels to the dataset)

### Visualizing the results of clustering

In [ ]:
fig, ax = plt.subplots()
markers = '^os+'
colors = ['r', 'g', 'b', 'Black']
for i, m in enumerate(markers):
    points = (y == i)
    c = [colors[l] for l in labels[points]]
    ax.scatter(X[points,0], X[points,1], c=c, marker=m)

In [ ]:
#Rearrange so that all same labels are consecutive
idx = np.argsort(labels)
rearranged_dists = euclidean_dists[idx,:][:,idx]
plot_distance_matrix(rearranged_dists)

## Deciding the number of clusters

### Using the error function

In [ ]:
error = np.zeros(11)
error[0] = 0
k_values = range(1,11)
for k in k_values:
    kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10)
    kmeans.fit_predict(X)
    error[k] = kmeans.inertia_

fig, ax = plt.subplots()
ax.plot(range(1,len(error)),error[1:])
ax.set_xlabel('Number of clusters')
ax.set_ylabel('Error')
ax.set_xticks(k_values)

### Silhouette Coefficient

Let $a$ be the mean distance between a sample and all other points in the same class and $b$ be the mean distance between a sample and all other points in the next nearest cluster. Then the 
**Silhoeutte Coefficient** for a clustering is:
$$s = \frac{b - a}{\max(a, b)}$$

In [ ]:
sc = metrics.silhouette_score(X, labels, metric='euclidean')
print(sc)

In [ ]:
def sc_evaluate_clusters(X, max_clusters):
    s = np.zeros(max_clusters+1)
    s[0] = 0;
    s[1] = 0;
    for k in range(2, max_clusters+1):
        kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10)
        kmeans.fit_predict(X)
        s[k] = metrics.silhouette_score(X, kmeans.labels_, metric='euclidean')
    fig, ax = plt.subplots()
    ax.plot(range(2,len(s)),s[2:])
    ax.set_xlabel('Number of clusters')
    ax.set_ylabel('Silhouette coefficient')
    ax.set_xticks(range(2, max_clusters+1))
    
sc_evaluate_clusters(X, 10)

## References

* [Boston University CS591 "Tools and Techniques for Data Mining and Applications"](https://github.com/dataminingapp/dataminingapp-lectures)
* https://github.com/yeseullee/Data-science-design-manual-notebooks
* [Zaki, 2014] *Data Mining and Analysis: Fundamental Concepts and Algorithms*, by Mohammed J. Zaki and Wagner Meira : Chapters 13, 14, and 17
* [Aggarwal, 2015] *Data Mining : The Textbook*, by Charu C. Aggarwal : Chapters 1 and 6 
* [Skiena, 2017] *The Data Science Design Manual*, by Steven S. Skiena